In [1]:
%load Macro.sas

In [ ]:
﻿/**************EXERCISE WITH MACROS!!!****************/

/*Exercise m102a04 -Using symbolgen option-*/

options symbolgen; /*each type we call a macro var is diplayed in the log!!!*/

%let type=Truck;
%let hp=250;

title1 "Car type: Truck";
title2 "Horepower>250";

proc print data=sashelp.cars;
var Make Model MSRP Horsepower;
where Horsepower>&hp and type = "&type";
run;

title;
options nosymbolgen;

/*Exercise m103a02 with %sysfunc()--->we can use data step function to manage macro variables!!!*/

%let dt=front;

data cars_subset;
	set sashelp.cars;
	where upcase(drivetrain) = %sysfunc(upcase("&dt"));
run;

title "%sysfunc(propcase(&dt))  Wheel Drive Cars";
footnote "Listing from %scan(&syslast,2) table";  /*syslast--> in this macro we memorize last dataset created*/

proc print data=&syslast;
run;
title;
footnote;

/*Exercise m103a03 with %str() --> this macro function mash special characters-->treated as test*/

%let city_all = %str(Rome, Italy);
%let city = %scan(&city_all,1,%str(,));
%put &=city;

/*Exercise m103p01 with %upcase %sysfunc %scan %symdel*/

%let FullName = JoNny CaLaMaRo;
%let FullName = %upcase(&FullName);

%put &FullName;

%let First = %sysfunc(propcase(%scan(&Fullname,1)));
%let Last = %sysfunc(propcase(%scan(&Fullname,2)));

%put &=FullName, &=First , &=Last;

%symdel FullName First Last;

%put _USER_;

/*Exercise m103p02 with %nrstr() --> equal to %str() + mask % and & (treated as text)*/

%let t = &systime;
%let d = &sysdate9;
%let Product = %nrstr(R&D);

title "Products contains &Product";
footnote "Report produced %sysfunc(today(),date9.) %sysfunc(time(),timeampm9.)";

proc print data=mc1.products;
	where Product_name contains "&Product";
	var Product_Name Product_ID Supplier_Name;
run;

/*Macro variables from an Sql query*/

/*m103p04 ex1*/

%let start= 01Feb2019;
%let end= 28Feb2019;

proc sql;
	select mean(quantity) format=4.2 as Qty, mean(total_retail_price) format=dollar7.2 as Price
	into :avgQuantity, :avgPrice
	from mc1.orders
	where order_date between "&start"d and "&end"d; 
quit;

title1 "Orders from &start to &end";
title2 "AvgQuantity: &avgQuantity , AvgPrice: &avgPrice";

proc print data= mc1.orders;
	where order_date between "&start"d and "&end"d;
	var Order_ID Order_Date Quantity Total_Retail_Price;
	sum Quantity Total_Retail_Price;
	format Total_Retail_Price dollar8.;
run;

/*m103p05 ex2*/

/*part1*/
%let year=2015;
%let basincode=EP;

proc sql noprint;
	select round(mean(MaxWindMPH)) as AvgWind
	into :AvgWind trimmed /*trimmed: in the put: AvgWind=92 and not AvgWind=     92*/
	from mc1.storm_final
	where season = &year and basin = "&basincode";
quit;

%put &=AvgWind;  

/*part2*/
title "&basincode storms in &year season";
title2 "MaxWind > season average of &AvgWind MPH";

proc print data=mc1.storm_final;
var Name StartDate EndDate MaxWindMPH MinPressure;
where MaxWindMPH > &AvgWind and Basin = "&basincode" and season = &year;
run;

title;

proc sgplot data=mc1.storm_final;
    where MaxWindMPH>&avgWind and Season=&year and Basin="&basincode"; 
	vbar StormType;
	yaxis display=(noline) grid;
run;

/*ex m103a04 on macro code execute before other code (into data set)*/

/*PRIORITY ORDER: Macro code statements (call macro variables, create macro variables) -> Compilation -->Execution*/

%let make=Honda;

data &make(keep=Make Model Type MSRP HybridFlag);
	set sashelp.cars end=lastrow;
	where upcase(Make) = %upcase("&make"); /*%upcase() macro fuction is runned before upset() function (execute during execution after compilation)*/
	retain HybridFlag;
	if Type = "Hybrid" then HybridFlag = 1;
	if lastrow = 1 then do;
		if HybridFlag = 1 then do;
			%let HybridText= There are Hybrid cars;  /*This code is execute before code in data set (in execution after compiling)*/
		end;
		else do;
			%let HybridText= There are not Hybrid cars; /*This code is execute before code in data set*/
		end;
	end;
run;

Title "&make cars";
footnote "&HybridText";  /*Is printed that there are no hybrid cars because macro statementes are execute before data set code is compiled/execute*/
proc print data = &make;
var make Model Type MSRP HybridFlag;
run;

title;
footnote;

/*Exercise m103a06 on Call symputx*/

/*call symputx is execute at execution time (after macro processor and compilation)*/

%let make= Honda;

proc means data=sashelp.cars noprint maxdec=0;
	where make = "&make";
	var MSRP;
	output out= CarsStat mean=mean;
run; 


data _null_;
set CarsStat;
call symputx("AvgMRSP",put(mean,dollar9.));
run;

title "&make Cars";
title2 "Average MRSP: &AvgMRSP";
proc print data=sashelp.cars;
where make = "&make";
run;

title;

/*Exercise m103p06 on Call symputx*/

%let dept = Sales;

data staff;
	set mc1.newhires end=lastrow;
	keep Employee_ID Department Job_Title Salary;
	where Department = "&dept";
	retain SumSalary;
	SumSalary + Salary;
	if lastrow then do;
	AvgSalary = SumSalary/_n_;
	call symputx("Avgsalary",put(AvgSalary,dollar9.));
	end;
run;

title "Department &dept";
footnote "Average salary: &AvgSalary";

proc print data=staff;
	sum salary;
run;

title;
footnote;

/*Exercise m103p07 on Call symputx*/

data _null_;
set mc1.storm_ocean_codes;
call symputx(Ocean,OceanName); /*We create n macro variables equal to the number of rows in mc1.Storm_ocean_codes*/
run;                           /*Name of the macro var equal to OCEAN and value of macro vars equal to OceanName*/

%put _user_;

proc print data=sashelp.vmacro;
	var name value;
	where length(name) = 1;  /*we print only macros with legth queal to 1*/
run;

/*Exercise m103p08 with indirect reference to macro variable*/

/*Indirect reference--> we call a macro var with another variable calling in first var call*/

%let code=3;

title "High Profit Product for &&type&code"; /*inderect reference*/

proc sql noprint;
select order_type
into :Type1-  /*Create 3 macro variables: Type1 Type2 Type3. Each macro var with a value of column order_type*/
from mc1.order_type_codes;
quit;


proc sql number;
select product_id format=z12. , sum(Total_retail_price) format= dollar10.2 as GrossSales,
	   sum(Total_Retail_Price - CostPrice_Per_Unit*Quantity) as Profit format= dollar10.2
from mc1.orders
where order_type=&code
group by Product_ID
having Profit/GrossSales >0.5
order by Profit desc;
quit;

title;	

/*Exercise m103p09 on indirect reference*/

%let code=ZA;

data _null_;
set mc1.country_codes;
call symputx(CountryCode,CountryName);
run;

title "Customers residing in &&&code";

/*Exercise with mcopilenote options*/

/*Mcopileoptions--> tells if there are errors when we define(creates) a macro*/

options mcompilenote=All;

%let dsn=sashelp.cars;
%let obs=5;

%macro PrintTable;
proc print data=&dsn(obs=&obs);
run;
%mend PrintTable;

options mcompilenote=none;

/*Exercise m104a02 on mcopilenote and mprint options*/

options mcompilenote=all;
options mprint; /*to verify the code inside the macro is correct when we call the macro program*/

%macro PrintTable;
proc print data=&dsn(obs=&obs);
run;
%mend PrintTable;

%let dsn = Sashelp.clas;
%let obs = 5;

%PrintTable

options mcompilenote=none;
options nomprint;

/*To see a list of macro created*/

proc catalog catalog=work.sasmacr;
	contents;
run;

/*Exercise m104a03 on macro parameters*/

options mcompilenote=all;

%macro stormchart(basin=NA, season=2016, maxwind=20);
title1 "Storm Frequency by type";
title2 "Basin &basin, Season: &season, Maxwind > &maxwind";
proc sgplot data=mc1.storm_final;
	vbar Stormtype / dataskin=pressed;
	where basin = "&basin" and season = &season and maxwindMPH > &maxwind;
run;
title;
%mend stormchart;

%stormchart()

%stormchart(basin=EP,season=2014)   /*maxwind use the standard value!!!*/

/*Exercise m104p01 with positional vs keyword parameter*/
options mcompilenote = all;
options mprint;
%macro customers(type=inactive);  /*macro customers(type)-->positional parameter,  macro customers(type=inactive)-->keyword parameter*/
%let type = %upcase(&type);
title "&type customers";
proc sql number;
select Name,Age_group,Type
from mc1.customers
where upcase(type) contains "&type";
quit;
title;
%mend customers;

%customers()

options mcompilenote=none;
options nomprint;

/*Exercise m104p02 on keyword parameters (default parameters)*/

options mcompilenote=all;
options mprint;

%macro ordersStats(data=mc1.orders,var=total_retail_price,stat=mean,class=customer_id,decimals=2);
title "&stat by &class for &var";
proc means data=&data &stat maxdec=&decimals;
var &var;
class &class;
title;
%mend ordersStats;

%ordersStats()

options mcompilenote=none;
options nomprint;

/*m104p03 with nrstr()*/

options mcompilenote=all;
options mprint;

%macro productList(prodline);
title "Listing of %sysfunc(propcase(&prodline))";
proc print data=mc1.products;
where upcase(product_line) = %upcase("&prodline");
var Product_ID Product_name Product_Line;
run;
%mend productList; 

%productlist(%nrstr(clothes&shoes))

/*Ex m104a04 Local macro variable vs Global macro variable*/

%let X=OutsideMacro;

%put NOTE: &=X before TEST macro execution;

%macro Test;
%let X=insideMacro;/*if a global macro variable exist a this let modify the global macro var*/
%put NOTE: &=X Inside TEST macro execution; 
%mend Test;

%Test

%put NOTE: &=X after TEST macro execution;

%put _global_;


/*A test to verify local variable!!!*/

%macro Prova;
%local X; /*with this statement also if a variable is global (a let statement outside a macro definition) it become local!!!*/
%let X = something;
%put _local_;
%mend Prova;

%prova

%put _global_;

/*Exercise m104a06 on parameters in a macro program*/

%let X=OutsideMacro;

%put &=X before the macro program;

%macro Test(X);
%global X; /*Gives an error because X as parameter is saved in a local symbol table*/
%put &X during the macro program;
%put _local_;
%mend Test;

%Test(Ciccio)

%put &=X after the macro program; /*it's equivalent to x before we run the macro program!!!-->x=OutsideMacro*/

/*m104p05 on local and global variables*/

%macro scope;
%let stormtype1=Some damage;      /*all variables are local!!!*/  
%let stormtype2=Extensive damage; 
%let stormtype3=Devastating damage;
%let stormtype4=Catastrophic damage;
%let stormtype5=Widespread catastrophic damage;
%put _user_; /*variable with name of macro before are local: SCOPE stormtype1 Some damage*/
%mend scope;

%scope

options mcompilenote=all;
options mprint;

%macro scope1;
proc sql noprint;  /*macro vars create by an sql step INSIDE a macro program are local*/
select damage
into: stormtype1- 
from mc1.storm_cat
order by category;
quit;
%let num = &sqlobs; /*sqlobs memorize the number of row of last sql query*/
%put _user_; 
%mend scope1;

%scope1

%macro scope2;
data _null_;
set mc1.storm_cat end=lastrow;   /*with macro vars created from a datasete INSIDE a macro program-->all vars are global!!*/
call symputx(cat('stormtype',_n_),damage);
if lastrow then call symputx('num',_n_);
run;
%put _user_;
%mend scope2;

%scope2

%symdel stormtype1 stormtype2 stormtype3 stormtype4 stormtype5 num;

%macro scope3;
data _null_;
%local X; /*if we create a local variables-->all other vars are local!!!*/
set mc1.storm_cat end=lastrow;   
call symputx(cat('stormtype',_n_),damage);
if lastrow then call symputx('num',_n_);
run;
%put _user_;
%mend scope3;

%scope3;

%symdel stormtype1 stormtype2 stormtype3 stormtype4 stormtype5 num;

/*Ex m104a07 with %DO statement*/

data sport;
set sashelp.cars;
where Type="Sports";
AVGMPG = mean(MPG_City,MPG_Highway);
run;


%if &SYSERR ne 0 %then %do;
	%put ERROR: There is an error in the data step;
%end;
%else %do;
	title "Sport cars";
	proc print data=sport;
	var msrp Model;
	run;
	proc sgplot data=sport;
	scatter x=msrp y=enginesize;
	run;
	title;
%end;

/*Exercise m104a08 with %do statements*/
options mcompilenote=all;
options mprint; /*debug the code in compilation/execution phase (after execution by macro processor)*/
options mlogic;  /*advise each time a macro var is called with relative value*/

%macro avgfuel(loc);
%if &loc= %then %do;
	%put ERROR: no parameter is entered for origin;
	%put ERROR: valid values: EUROPE , USA, ASIA;
	%return;
%end;
%else %if %upcase(&loc) = USA %then %do;
	%let loc = %upcase(&loc);
	data fuel_&loc;
	set sashelp.cars;
	where upcase(origin) = "&loc";
	AVGMPG= mean(MPG_City,MPG_highway);
	keep make model type avgmpg;
%end;
%else %if %upcase(&loc) = ASIA or %upcase(&loc) = EUROPE %then %do;
	%let loc = %upcase(&loc);
	data fuel_&loc;
	set sashelp.cars;
	where upcase(origin) = "&loc";
	AVGKmL = mean(MPG_City,MPG_highway)*0.425;
	keep make model type avgkml;
%end;
%else %do;
	%put ERROR: you entered wrong value;
	%put ERROR: valid values: EUROPE , USA, ASIA;
	%return;
%end;
title "Fuel efficenty";
title2 "Origin: &loc";
proc print data=fuel_&loc;
	run;
title; 
%mend avgfuel;

%avgfuel(Europe)

%avgfuel(Usa)

%avgfuel(Pino)

/*Exercise m104a09 on %DO LOOP*/

%macro allcylinders(start,stop);
%do num=&start %to &stop;
	title "&num-cilinders Cars";
	proc print data=sashelp.cars noobs;
	var Make Model Type Origin MSRP MPG_City MPG_Highway;
	where Cylinders=&num;
	run;
	%put _user_;
%end;
title;
%mend allcylinders;

%allcylinders(3,12)

/*Exercise m104p10 on %do loop and indirect macro var reference*/

%macro Orders;
%do num=1 %to 3;
	title "order type: &num";
	proc means data=mc1.orders sum mean maxdec=2;
	where order_type=&num;
	var Total_Retail_Price CostPrice_Per_Unit;
	run;
%end;
%mend Orders;

%Orders

%macro Orders2;
	data _null_;
	set mc1.order_type_codes end=last;
	call symputx(cat("type",_n_),Order_type);
	if last then call symputx("numTypes", _n_);
	run;
%do i=1 %to &numTypes;
	title "Order Type: &&type&i";
	proc means data=mc1.orders sum mean maxdec=2;
	where Order_Type=&i;
	var Total_Retail_Price CostPrice_Per_Unit;
	run;
%end;
%mend Orders2;

%orders2

/*List of our macro program created with  filter (where clause)!!!*/

%macro PrintTable(dsn=sashelp.cars,obs=5);
proc print data=&dsn(obs=&obs);
run;
%mend PrintTable;

proc catalog catalog=work.sasmacr;
contents;
contents out=macrolist(where=(name like 'ORDER%'));
run;

/*Autocall library*/

/*Autocall library--> a library with sas files where in each sas file there is a macro program definition*/
                /*--> macro definition can be already definied by SAS or by user*/

%put NOTE: sasautos =%sysfunc(pathname(sasautos)); /*a list of path with sas programs with macro definition inside*/
                                                   /*sasautos is a filef that points to paths where macro program supplied by SAS are stored*/

/*Exercise m105a02 on when autocall macro function are added*/
options nomlogic;
options nomprint;

proc catalog catalog=work.sasmacr;
contents;
run;

%let test=abc;
%let number=123;

%put The type of test is %datatyp(abc);
%put The type of number is %datatyp(number);

proc catalog catalog=work.sasmacr;  /*%datatyp is added (with new sas macro program to the table) */
contents;                           /*This new macro programs are added, compiled and executed */

/*Adding a personal macro to autocall library*/

options mautosource;  /*the macro process is able to search */
                      /*the autocall libraries when the requested macro program isn't found in work.sasmacr*/

options sasautos=("&path/autocall",sasautos);  /*we add another autocall library, second one is the standard filef provided by SAS*/

%stormchart  /*this macro program is called from our autocall library (path: &path/autocall)*/


/*Exercise m105a03 on calling user created macro program*/

%put %propcase(Ciao pinello); /*Its work!! call an user created macro program saved in &path/autocall (autocall library)*/

/*Exercise m105a04 with create a macro var with more than one value from a sql query*/

%let tab=sashelp.cars;
%let col=Type;

proc sql;
select distinct &col
into :tablist separated by " "
from &tab; 
quit;

%put &tablist;

/*Exercise m105p02 on creating a macro program with macro vars from sql query*/

%macro GroupList(Tab,Col);
proc sql noprint;
select distinct upcase(&col)
into :Val1-
from &tab
quit;
%do i=1 %to &Sqlobs;
title "Group &&Val&i";
proc print data=&tab;
where upcase(type)= "&&Val&i";
run;
%end;
%mend GroupList;

%grouplist(sashelp.cars,type)

/*Exercise m105a05 on macro program with sql statement*/

%macro splittable(tab,col);
	proc sql noprint;
	select distinct &col
	into :var1-
	from &tab;
	quit;
	data
	%do i=1 %to &sqlobs;
		&&var&i;
	%end;
		set &tab;
		select(&col);
	%do i=1 %to &sqlobs;
		when ("&&var&i") output &&var&i;
	%end;
	otherwise;
	end;
	run;
%mend splittable;

%splittable(sashelp.cars,origin)

%splittable(sashelp.cars,Price)

%splittable(sashelp.cars,DriveTrain) 


/*Exercise m105a06 on dictionary.columns*/

options nolabel;

proc sql;
select name
into :columnList1 separated by ","
from dictionary.columns
where libname=upcase("sashelp") and memname=upcase("cars");  /*sashelp-->library, cars-->table*/
quit;

options label;

%put Column of Cars: &columnList1;

/*Exercise m105p04 on macro program that manage errors*/

options minoperator /*you can use not in %if statement*/

%macro customerType(type);
	%let type = %upcase(&type);
	%if &type= %then %do;
	%put ERROR: Missing type;
	%put ERROR: Valid type are HIGH, MEDIUM, LOW;
	%end;
	%else %if not (&type in HIGH MEDIUM LOW) %then %do;
	%put ERROR: Invalid TYPE: &type;
	%put ERROR: Valid type are HIGH, MEDIUM, LOW;
	%end;
	%else %do;
	title "&type Activy customers by country";
	proc freq data=mc1.customers order=freq;
	where upcase(type) contains "&type";
	tables country;
	%end;
%mend customerType;

%customerType(high)

%customerType()  /*parameter as missing value give a specific error*/

%customerTYpe(lower)

